# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_purchasings = pd.DataFrame(data.groupby(['CustomerID','ProductName']).Quantity.sum())
df_purchasings.head(20)

Quantity
CustomerID ProductName                              
33         Apricots - Dried                        1
           Assorted Desserts                       1
           Bandage - Flexible Neon                 1
           Bar Mix - Pina Colada, 355 Ml           1
           Beans - Kidney, Canned                  1
           Beef - Chuck, Boneless                  1
           Beef - Prime Rib Aaa                    1
           Beer - Original Organic Lager           1
           Beer - Rickards Red                     1
           Black Currants                          1
           Blackberries                            1
           Bread - Italian Roll With Herbs         1
           Bread Foccacia Whole                    1
           Cake - Box Window 10x10x2.5             1
           Cassis                                  1
           Cheese - Cottage Cheese                 1
           Crab - Imitation Flakes                 1
           Fondant - Icing                         1
           French Pastry - Mini Chocolate          1
           General Purpose Trigger                 1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df_purchasings.fillna(0)

matrix = pd.pivot_table(df_purchasings, values = 'Quantity', index = ['ProductName'] , columns = ['CustomerID'])
matrix1 = matrix.fillna(0)
matrix1.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(matrix1.T, 'euclidean'))), 
                         index=matrix1.columns, columns=matrix1.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = distances[200].sort_values(ascending=False)[1:] #cliente ID=200
top5 = similarities.head().index.to_list()
top5

[3317, 1072, 3535, 1920, 3909]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
df = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'}).reset_index()
select = df[df['CustomerID'].isin(top5)]
select.head()

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
#q quantity
quantity = select.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
quantity.head()

,Quantity
ProductName,
Soup - Campbells Bean Medley,4
Muffin - Carrot Individual Wrap,3
Bay Leaf,3
Pork - Kidney,3
"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
#Not prchased
not_purchased = matrix1[matrix1[200]==0]
#Merge
m = not_purchased.merge(quantity, on='ProductName').sort_values('Quantity',ascending=False)
display(m[[200, 'Quantity']].head())

,200,Quantity
ProductName,,
Soup - Campbells Bean Medley,0.0,4
Muffin - Carrot Individual Wrap,0.0,3
Wanton Wrap,0.0,3
Bay Leaf,0.0,3
Pork - Kidney,0.0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
#recomendaciones

#Empty array
reco={}

#List of unique CustomerIDs
cust=list(data['CustomerID'].unique())

#Iterate over the list
for i in cust:
    #Definig the top 5 similarities of each i
    similarities = distances[i].sort_values(ascending=False)[1:] #ID i
    top5 = similarities.head().index.to_list()
    
    #Selecting the top 5 ID´s from the mean dataframe in step 1 Dataframe grouped by CustomerID and ProductName
    select = df.loc[df['CustomerID'].isin(top5)]
    similar = select.groupby(['ProductName']).agg({'Quantity':'sum'}).reset_index()
    
    #Creating the merge to selec the top recomended products
    m = similar.merge(matrix1, on='ProductName')
    not_purch = m[m[i]==0]
    
    #Selecting only the first 5 products based on the most purchased 
    quantity = not_purch.sort_values('Quantity', ascending=False)['ProductName'][0:5].tolist()
    reco.update({i: quantity})


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
top_recom = pd.DataFrame.from_dict(reco, orient='index').reset_index()
top_recom.columns=['CustomerID','Recom1','Recom2','Recom3','Recom4','Recom5']
top_recom.head()

,CustomerID,Recom1,Recom2,Recom3,Recom4,Recom5
0,61288,Chicken - Soup Base,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural"
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Pernod,Wanton Wrap,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(matrix1.T, 'correlation'))), 
                         index=matrix1.columns, columns=matrix1.columns)
#recomendaciones

#Empty array
reco={}

#List of unique CustomerIDs
cust=list(data['CustomerID'].unique())

#Iterate over the list
for i in cust:
    #Definig the top 5 similarities of each i
    similarities = distances2[i].sort_values(ascending=False)[1:] #ID i
    top5 = similarities.head().index.to_list()
    
    #Selecting the top 5 ID´s from the mean dataframe in step 1 Dataframe grouped by CustomerID and ProductName
    select = df.loc[df['CustomerID'].isin(top5)]
    similar = select.groupby(['ProductName']).agg({'Quantity':'sum'}).reset_index()
    
    #Creating the merge to selec the top recomended products
    m = similar.merge(matrix1, on='ProductName')
    not_purch = m[m[i]==0]
    
    #Selecting only the first 5 products based on the most purchased 
    q = not_purch.sort_values('Quantity', ascending=False)['ProductName'][0:5].tolist()
    reco.update({i: q})

top_recom = pd.DataFrame.from_dict(reco, orient='index').reset_index()
top_recom.columns=['CustomerID','Recom1','Recom2','Recom3','Recom4','Recom5']
top_recom.head()


,CustomerID,Recom1,Recom2,Recom3,Recom4,Recom5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Fondant - Icing,Grenadine,Mangoes,Lettuce - Frisee,Veal - Osso Bucco
2,40094,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
3,23548,Olives - Stuffed,Wine - Charddonnay Errazuriz,Soup - Campbells Tomato Ravioli,Assorted Desserts,Juice - Orange
4,78981,"Tart Shells - Sweet, 4",Artichokes - Jerusalem,Sprouts - Baby Pea Tendrils,"Appetizer - Mini Egg Roll, Shrimp",Chocolate - Dark
